## What to do
Implement a simple algorithm on **Text Segmentation**:
- Use as a test an input of k paragraphs taken from different topics (e.g. Wikipedia pages)
- Is our system able to find the right cuts?

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from collections import Counter
from random import randint
from typing import Set, List, Tuple

### Data Load

In [2]:
def import_corpora() -> List[str]:
    with open('data/topics_corpora.txt', encoding='utf-8') as f:
        sentences = f.read().splitlines()
    return sentences

corpora = import_corpora()
stop_words = set(stopwords.words('english'))

### Pre-processing

In [3]:
# Pre-processing of a sentence
def bag_of_words(sentence: str) -> Set[str]:
    return set(remove_stopwords(tokenize_sentence(remove_punctuation(sentence))))


# Remove stopwords from a word list
def remove_stopwords(words: List[str]) -> List[str]:
    return [value.lower() for value in words if value.lower() not in stop_words]


# Get tokens from sentence
def tokenize_sentence(sentence: str) -> List[str]:
    words = []
    lmtzr = WordNetLemmatizer()
    for tag in nltk.pos_tag(word_tokenize(sentence)):
        words.append(lmtzr.lemmatize(tag[0]).lower())
    return words


# Remove punctuation and multiple spaces
def remove_punctuation(sentence: str) -> str:
    return re.sub('\s\s+', ' ', re.sub(r'[^\w\s]', '', sentence))

### 30 Most Common Words Computation

In [4]:
n_MCW = 30 # num of most common words

counter = Counter()
for sentence in corpora:
    counter.update(bag_of_words(sentence))

counter.most_common(n_MCW)

[('film', 49),
 ('football', 45),
 ('wa', 39),
 ('computer', 32),
 ('first', 27),
 ('image', 26),
 ('game', 26),
 ('music', 24),
 ('camera', 23),
 ('science', 22),
 ('color', 21),
 ('field', 20),
 ('used', 18),
 ('also', 17),
 ('one', 17),
 ('ball', 17),
 ('system', 16),
 ('software', 15),
 ('played', 15),
 ('movie', 15),
 ('digital', 14),
 ('known', 14),
 ('computation', 13),
 ('using', 13),
 ('many', 13),
 ('picture', 13),
 ('theory', 12),
 ('data', 12),
 ('process', 12),
 ('association', 12)]

### Cuts Computation

In [5]:
# Return |n_cuts| random indices within the max_value range
def random_cuts(n_cuts: int, max_value: int) -> List[int]:
    cuts = [randint(0, max_value-1) for _ in range(n_cuts-1)]
    return sorted(cuts)


# Co-occurrence of a segment is measured as the sum of all the occurrences of the n_MCW most common words
def co_occurrence(segment: List[str], n_MCW: int) -> int:
    score = 0
    counter = Counter()
    for sentence in segment:
        counter.update(bag_of_words(sentence))
    
    most_common_words = counter.most_common(n_MCW)
    score = sum([entry[1] for entry in most_common_words])
    return score


# Compute score for a list of segments deriving from a list of cuts
def compute_score(cuts: List[int], corpora: List[str], n_MCW: int) -> int:
    scores = []

    # in this loop we want to compute segments score based on cuts
    # es. cuts: [40, 145, 238] --> segments: corpora[0, 40], corpora[40, 145], corpora[145, 238], corpora[238, len(corpora) - 1]
    for j in range(len(cuts) + 1):
        start = cuts[j-1] if j > 0 else 0
        end = cuts[j] if j < len(cuts) else len(corpora) - 1
        segment = corpora[start:end]
        
        score = co_occurrence(segment, n_MCW)
        scores.append(score)
    
    return sum(scores)
    

# Core function, it search the optimal cuts to maximize the score (co_occurrence) of each segment
def maximize_score(corpora: List[str], n_MCW: int, n_topics: int, max_iterations: int, adjustment_range: int = 10) -> Tuple[int, List[int]]:
    max_score = 0
    best_cuts = None

    # Bruteforce phase
    for i in range(max_iterations):
        # Compute random cuts
        cuts = random_cuts(n_topics, len(corpora)-1)
        # Compute score for the cuts obtained
        current_score = compute_score(cuts, corpora, n_MCW)
        if current_score > max_score:
            max_score = current_score
            best_cuts = cuts
            print(f"completed iteration {i+1} - max_score: {max_score}, best_cuts: {best_cuts}")
    
    # Refine phase
    cuts = best_cuts.copy()
    for i in range(len(cuts)):
        # Refine the cuts obtained from the bruteforce phase using an adjustment range
        for adjustment in range(-adjustment_range//2, adjustment_range//2):
            cuts[i] += adjustment
            # Compute score for the cuts obtained
            current_score = compute_score(cuts, corpora, n_MCW)
            if current_score > max_score:
                max_score = current_score
                best_cuts = cuts
                print(f"refining cuts - max_score: {max_score}, best_cuts: {best_cuts}")

    return max_score, best_cuts

In [6]:
n_MCW = 3 # num of most common words
n_topics = 4 # total topics in the corpora
max_iterations = 1000 # total loops of the bruteforce phase
adjustment_range = 5 # range to refine the cuts

max_score, best_cuts = maximize_score(corpora, n_MCW, n_topics, max_iterations, adjustment_range=adjustment_range)
print(f'Max score is: {max_score}')
print(f'Best cuts indices are: {best_cuts}')

completed iteration 1 - max_score: 246, best_cuts: [68, 163, 299]
completed iteration 12 - max_score: 253, best_cuts: [29, 62, 154]
completed iteration 47 - max_score: 254, best_cuts: [56, 156, 279]
completed iteration 64 - max_score: 258, best_cuts: [67, 111, 184]
completed iteration 121 - max_score: 262, best_cuts: [56, 158, 218]
completed iteration 242 - max_score: 270, best_cuts: [67, 168, 257]
completed iteration 809 - max_score: 279, best_cuts: [58, 145, 258]
Max score is: 279
Best cuts indices are: [58, 145, 258]
